In [1]:
rqdatac_password="Z5tTdXcy3x539LsXpmiemHQSx_hj0kKM_SdCSi_uOm6S91N-1Hd_F6DSUb-TmxqCbNGa-nJEOtAgNp6RdpXje37YiioD5EuMtLb0vkyKaqKnVawJIy2OwLq_1Fih9kbSn6ecRD_IVYrO6R2gGVXHbkHSpZ45XYNZ4gukTYpEZI8=CKgqIT0qkhsDUftw4Ohq7ZJYJE29WpuKU8JSjQapbLAQ8PuDLhezOy5nLWiSj1wJ0Dt2PoRxW5hapHKQlXqw3IpQ2kw0Ozu4IXTwKVaPFxKm1eXIE3xcJ8tNlNu89X6sPIJ_B7w1mu6tF3f3DNU6euV__zYpD_h2ZEGI7imK-yk="

In [2]:
host="rqdatad-pro.ricequant.com"
port="16011"
address=(host, port)

In [3]:
import rqdatac as rq
rq.init(username="license", password=rqdatac_password, addr=address)
from rqdatac import *

In [4]:
import numpy as np
import pandas as pd
import os
import pickle

In [7]:
get_price('T1903',start_date='2018-12-10', end_date='2018-12-10', frequency='1d')

,close,high,settlement,open,low,limit_up,volume,open_interest,prev_settlement,total_turnover,limit_down
2018-12-10,97.285,97.41,97.29,97.34,97.215,99.055,35563.0,62851.0,97.115,3.460980e+10,95.175


In [17]:
futures.get_member_rank('T1903',trading_date=20181210,rank_by='short')

,commodity_id,member_name,rank,volume,volume_change
trading_date,,,,,
2018-12-10,T1903,中信期货,1,8595,626
2018-12-10,T1903,国泰君安,2,7660,1
2018-12-10,T1903,永安期货,3,6105,-59
2018-12-10,T1903,银河期货,4,5848,22
2018-12-10,T1903,海通期货,5,5749,-92
2018-12-10,T1903,广发期货,6,3696,37
2018-12-10,T1903,国金期货,7,2610,-217
2018-12-10,T1903,宝城期货,8,1935,-37
2018-12-10,T1903,国海良时,9,1686,44


In [13]:
futures.get_member_rank('T1903',trading_date=20181210,rank_by='long')

,commodity_id,member_name,rank,volume,volume_change
trading_date,,,,,
2018-12-10,T1903,国泰君安,1,6693,-58
2018-12-10,T1903,中信期货,2,4907,-77
2018-12-10,T1903,银河期货,3,4732,64
2018-12-10,T1903,浙商期货,4,3627,46
2018-12-10,T1903,中金期货,5,3258,3
2018-12-10,T1903,海通期货,6,3193,-96
2018-12-10,T1903,新湖期货,7,2574,31
2018-12-10,T1903,广发期货,8,2558,35
2018-12-10,T1903,永安期货,9,2464,-135


In [5]:
raw_data_path='../data/raw/'

# Treasury Futures Contract Info

In [6]:
all_contracts_info=all_instruments(type='Future')
interim_contracts_info=all_contracts_info[all_contracts_info['order_book_id'].str.match('T' or 'TF' or 'TS')]
treasury_futures_contracts_info=interim_contracts_info[~interim_contracts_info['order_book_id'].str.contains('TA|TC|88|99')]
treasury_futures_contracts_info=treasury_futures_contracts_info.set_index('order_book_id').sort_index()

In [8]:
# Remove existing expiry_date csv file
if os.path.isfile(raw_data_path+'meta_data/'+'treasury_futures_contracts_info.csv'):
    os.remove(raw_data_path+'meta_data/'+'treasury_futures_contracts_info.csv')
    
# Save new expiry date data into the csv file
if not os.path.exists(raw_data_path+'meta_data'):
    os.makedirs(raw_data_path+'meta_data')
treasury_futures_contracts_info.to_csv(raw_data_path+'meta_data/'+'treasury_futures_contracts_info.csv', encoding="gb2312")

# Download Historical Data

In [9]:
daily_data_path='../data/raw/daily_data/'

In [10]:
start_date='2013-01-01'
end_date='2019-07-15'
cols=['open', 'high', 'low', 'close', 'settlement', 'prev_settlement', 'limit_up', 'limit_down', 'volume', 'total_turnover', 'open_interest']

In [11]:
treasury_futures_contracts_list=list(treasury_futures_contracts_info.index)

In [11]:
for contract_code in treasury_futures_contracts_list:
    contract=contract_code[:-4]
    listed_date=instruments(contract_code).listed_date
    de_listed_date=instruments(contract_code).de_listed_date
    if de_listed_date>end_date:
        if end_date>=listed_date:
            df=get_price(contract_code, start_date=listed_date, end_date=end_date, frequency='1d', fields=cols)
            if df is not None:
                df=df[cols]
                df.insert(0, 'contract_code', contract_code)
                df.columns=['Contract_Code', 'Open', 'High', 'Low', 'Close', 'Settle', 'Prev Settle',\
                            'Limit Up', 'Limit Down', 'Volume', 'Turnover', 'Open_Interest']
                df.index.name='Trading_Day'
    else:
        df=get_price(contract_code, start_date=listed_date, end_date=de_listed_date, frequency='1d', fields=cols)
        if df is not None:
            df=df[cols]
            df.insert(0, 'contract_code', contract_code)
            df.columns=['Contract_Code', 'Open', 'High', 'Low', 'Close', 'Settle', 'Prev Settle',\
                        'Limit Up', 'Limit Down', 'Volume', 'Turnover', 'Open_Interest']
            df.index.name='Trading_Day'
            
    if df is not None:
        csv_file=contract_code+'.csv'
        if not os.path.exists(daily_data_path+'/'+contract):
            os.makedirs(daily_data_path+'/'+contract)
        df.to_csv(daily_data_path+'/'+contract+'/'+csv_file)

In [17]:
minute_data_path='../data/raw/minute_data/'
cols=['open', 'high', 'low', 'close', 'volume', 'total_turnover', 'open_interest']

In [19]:
for contract_code in treasury_futures_contracts_list:
    contract=contract_code[:-4]
    listed_date=instruments(contract_code).listed_date
    de_listed_date=instruments(contract_code).de_listed_date
    if de_listed_date>end_date:
        if end_date>=listed_date:
            df=get_price(contract_code, start_date=listed_date, end_date=end_date, frequency='5m', fields=cols)
            if df is not None:
                df=df[cols]
                df.insert(0, 'contract_code', contract_code)
                df.columns=['Contract_Code', 'Open', 'High', 'Low', 'Close',  'Volume', 'Turnover', 'Open_Interest']
                df.index.name='Trading_Day'
    else:
        df=get_price(contract_code, start_date=listed_date, end_date=de_listed_date, frequency='5m', fields=cols)
        if df is not None:
            df=df[cols]
            df.insert(0, 'contract_code', contract_code)
            df.columns=['Contract_Code', 'Open', 'High', 'Low', 'Close', 'Volume', 'Turnover', 'Open_Interest']
            df.index.name='Trading_Day'
            
    if df is not None:
        csv_file=contract_code+'.csv'
        if not os.path.exists(minute_data_path+'/'+contract):
            os.makedirs(minute_data_path+'/'+contract)
        df.to_csv(minute_data_path+'/'+contract+'/'+csv_file)